# Initialize libraries

In [1]:
from datetime import datetime
import os
import glob

import deeplabcut
import pandas as pd

from dlchelperclass import DlcHelperClass as dhc

Loading DLC 3.0.0rc5...


# Create new project

In [ ]:
# Enter your project name
project_name = ''
# If you want to use labeled frames from an earlier project,
# make sure to use the same experiment name as in the previous project.
# Enter your experimenter name
experimenter_name = ''
# Specify where you want your project folder to be created
work_dir = ''
# create project path
project_path = work_dir + '/' + project_name + '-' + experimenter_name + '-' + datetime.now().strftime("%Y-%m-%d")
# video path
videos_dir_path = ''
# Creates a list variable of all videos paths 
video_path = dhc.get_video_paths(path=videos_dir_path)
# Create project
deeplabcut.create_new_project(
    project=project_name,
    experimenter=experimenter_name,
    videos=video_path,
    working_directory=work_dir,
    copy_videos=True
)

# Config yaml path
config_path = project_path + '/config.yaml'

# Initialize paths (For loading existing project)
# skip this cell if you created a new project

In [ ]:
# Enter your project path
project_path = ''
# Creates path for config file and videos
config_path, video_path = dhc.get_config_and_video_paths(project_path=project_path)

# Extract frames

In [ ]:
# Extract frames for labeling, amount of images and train/test ratio defined in config.yaml

# "Do you want to extract (perhaps additional) frames for video:" answer yes

deeplabcut.extract_frames(config=config_path, mode='automatic')

# Use napari for labeling 

# Creates CSV and h5 files needed for training

In [ ]:
deeplabcut.create_training_dataset(config=config_path)

# Train network

In [ ]:
deeplabcut.train_network(config_path)

# Creates training_stats.csv

In [ ]:
# Needed for plotting losses
deeplabcut.evaluate_network(config_path, plotting=True)

# Makes prediction on videos

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

# Create new videos with the models predicted labels

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

# Evaluate models

In [ ]:
# Creates likelihood.txt file and loss/vall_loss plot
dhc.save_mean_likelihood_to_file(project_path=project_path)
dhc.plot_loss_to_png(project_path)

# Extract outlier frames and re-labeling

In [ ]:
deeplabcut.extract_outlier_frames(config=config_path,
                                   shuffle=1,
                                  automatic=True,
                                  videos=video_path)

In [ ]:
# Open the DeepLabCut GUI to manually correct labels
# Launch the GUI to manually adjust labels on the extracted frames
deeplabcut.refine_labels(project_path)

In [ ]:
deeplabcut.merge_datasets(config=config_path)

In [ ]:
# Create a new training dataset that includes the updated labels
# This prepares the training files with the corrected labels for training
deeplabcut.create_training_dataset(config=config_path)

# Finetune 

In [ ]:
# Make sure you have run the following code line before:
# * deeplabcut.analyze_videos(project_path, video_paths)

In [ ]:
# Retrain the model with the updated dataset
# Use transfer learning to improve the existing model with the new data
deeplabcut.train_network(project_path, shuffle=1, displayiters=100, saveiters=1000)

In [ ]:
# Re-evaluate performance on the updated model
# Analyze the videos again to check if model accuracy has improved after retraining
deeplabcut.analyze_videos(project_path, video_path)

# Predict unseen data

In [ ]:
video_path = ''

deeplabcut.analyze_videos(config_path, videos=video_path)

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

# If several videos to predict with one model 
# Analyze and label videos and get h5 file

In [ ]:
# Predict several videos and models

# your project folder path
base_path = ''

# your project folder from DLC there you have your .yaml file
paths = [
    '',
]

for path in paths:
    project_path = base_path + path.strip()  
    
    config_path, video_path = dhc.get_config_and_video_paths(project_path=project_path)

    # your folder there you have your video you want to predict
    video_path = ''

    # Assuming your videos have extensions like .mp4, .avi, etc.
    video_extensions = ('*.mp4', '*.avi', '*.mkv')  # Add other extensions if needed
    video_files = []

    for ext in video_extensions:
        video_files.extend(glob.glob(os.path.join(video_path, ext)))

    for video_file in video_files:
        # Analyze video    
        deeplabcut.analyze_videos(config_path, video_file)
        # Create labels
        deeplabcut.create_labeled_video(config_path, videos=video_file)

# Get likelihood from one predicted video printed out 

In [ ]:
# Path to your h5 file
h5_path = ''
df = pd.read_hdf(h5_path)
df.columns = [f"{bodypart}_{coord}" for bodypart, coord in zip(df.columns.get_level_values(1), df.columns.get_level_values(2))]
df_monofil = df.loc[:, df.columns.str.startswith(('FR', 'FG', 'FB')) & ~df.columns.str.endswith('likelihood')]
df_square = df.loc[:, df.columns.str.startswith(('Top_left', 'Top_right', 'Bottom_left', 'Bottom_right')) & ~df.columns.str.endswith('likelihood')]
df_likelihoods = df.loc[:, df.columns.str.endswith('likelihood')]
overall_average = df_likelihoods.mean().mean()
print("Overall Average Likelihood:", overall_average)
bodypart_means = df_likelihoods.mean(axis=0)
print("Mean likelihood for each body part:")
print(bodypart_means)

# Likelihood predicted Videos in txt file
# Possible with several videos 

In [ ]:
def process_h5_files(directory, output_file):
    # Find all .h5 files in the specified directory
    h5_files = glob.glob(os.path.join(directory, '*.h5'))
    
    total_sum = 0  # Initialize total sum of overall_average values
    file_count = len(h5_files)  # Number of files processed
    
    with open(output_file, 'w') as f_out:
        for h5_path in h5_files:
            # Read the .h5 file into a DataFrame
            df = pd.read_hdf(h5_path)
            df.columns = [
                f"{bodypart}_{coord}" 
                for bodypart, coord in zip(
                    df.columns.get_level_values(1), 
                    df.columns.get_level_values(2)
                )
            ]
            
            # Select likelihood columns
            df_likelihoods = df.loc[:, df.columns.str.endswith('likelihood')]
            
            # Calculate the overall average likelihood
            overall_average = df_likelihoods.mean().mean()
            
            # Add overall_average to total_sum
            total_sum += overall_average
            
            # Calculate the mean likelihood for each body part
            bodypart_means = df_likelihoods.mean(axis=0)
            
            # Write the results to the output file
            f_out.write(f"File: {os.path.basename(h5_path)}\n")
            f_out.write(f"Overall Average Likelihood: {overall_average}\n")
            f_out.write("Mean likelihood for each body part:\n")
            f_out.write(bodypart_means.to_string())
            f_out.write("\n")
            f_out.write("-" * 37 + "\n")  # Separator line
        
        # Calculate the overall mean across all files
        overall_mean = total_sum / file_count if file_count > 0 else 0
        
        # After processing all files, write only the overall mean
        f_out.write("\n\n\nOverall Mean of All Videos: {:.2f}\n".format(overall_mean))

# Example usage:
directory = ''  # Replace with your directory path
output_file = 'likelihoods_predict.txt'
process_h5_files(directory, output_file)
